In [1]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer  # binaryclass
import cv2 as cv
import os
from Layer import Layer
from NN import NN
import numpy as np
from sys import path
path.append('..')
import imutils

In [2]:
### Import data

path_human = 'Human-2'
path_non_human = 'Non-Human-2'
ones_human = np.ones((64*64, 1)).reshape(1, -1)

# convert image shape to 1x4096 and put into a matrix ones_human

for file in os.listdir(path_human):
    path_img_human = path_human+'/'+file
    image_human = cv.imread(path_img_human, 0).reshape(1, -1)
    ones_human = np.vstack((ones_human, image_human))

data_human = np.delete(ones_human, 0, 0)

# N = number of data set, d is the number of features

N, d = data_human.shape
# print(N, d)


ones_non_human = np.ones((64*64, 1)).reshape(1, -1)
for file in os.listdir(path_non_human):
    path_img_non_human = path_non_human+'/'+file
    image_non_human = cv.imread(path_img_non_human, 0).reshape(1, -1)
    ones_non_human = np.vstack((ones_non_human, image_non_human))

data_non_human = np.delete(ones_non_human, 0, 0)

feature_set = np.vstack((data_human, data_non_human))

label_ones = np.ones((data_human.shape[0], 1))
label_zeros = np.zeros((data_non_human.shape[0], 1))
targets = np.vstack((label_ones, label_zeros))

X_train, X_test, Y_train, Y_test = train_test_split(
    feature_set, targets, test_size=0.2)


In [3]:
### train the network
### skip this step if already have the pre-trained weights

# create the network
nn_model = NN(X_train, Y_train)
nn_model.add_layer(Layer(50, activation='relu'))
nn_model.add_layer(Layer(25, activation='sigmoid'))


# fit the network
nn_model.fit(iteration=20000, learning_rate=0.001)

# plot cost function

Y_train_pred = nn_model.predict(X_train)
Y_test_pred = nn_model.predict(X_test)

accuracy_train = np.sum(Y_train_pred == Y_train) / len(Y_train) * 100
accuracy = np.sum(Y_test_pred == Y_test) / len(Y_test) * 100
print("Train accuracy: " + str(accuracy_train) + "%")
print("Test accuracy: " + str(accuracy) + "%")

0.6943929853834523
0.6943808849322697
0.6943680768806955
0.6943429711590927
0.6943160210664197
0.6942872441274563
0.694259233200339
0.6942307569564801
0.6942018502148324
0.69417209646151
0.694141934488258
0.6941108086524573
0.6940787665560544
0.6940455960860372
0.6940112273996683
0.6939754533905842
0.6939381768765005
0.6938992154389636
0.6938584102945184
0.6938156516520675
0.6937707166754259
0.6937234741413666
0.6936736509305673
0.6936210803820858
0.6935654286212839
0.693506598515568
0.6934444034238082
0.6933785387581044
0.6933087297138305
0.6932347401478616
0.6931562755157927
0.6930730199923986
0.6929846660339388
0.6928908625593501
0.6927912499154447
0.6926855399854919
0.6925731922971752
0.6924539283012401
0.692327282931684
0.6921928175996044
0.6920501234125473
0.6918987470124458
0.6917381131475906
0.6915677798644715
0.6913872486330486
0.6911960643381747
0.6909937249336735
0.6907797539773624
0.6905534060517637
0.6903145300465887
0.690062383259146
0.6897966853853177
0.6895169687570482


0.3973089983313168
0.39664423718261954
0.39598146957530966
0.3953207153147152
0.39466199333268076
0.3940053215980436
0.39335071708012703
0.39269819567356207
0.3920477721741687
0.39139946021376276
0.3907532722467809
0.3901092195132325
0.38946731203432583
0.38882755858788576
0.3881899667177083
0.3875545427290287
0.38692129192814156
0.3862902183389908
0.38566132441825723
0.38503461170656195
0.38441008061977977
0.38378773049905934
0.38316755964085303
0.3825495653510279
0.3819337440066642
0.3813200910702606
0.3807086011132884
0.38009926793714294
0.37949208458860173
0.3788870434157488
0.3782841361219494
0.3776833538380895
0.3770846870488774
0.3764881257819854
0.3758936609240256
0.3753012880997053
0.374710988123545
0.3741227494169635
0.3735365600707984
0.37295240793386714
0.3723702807469045
0.37179016722195884
0.3712120530965824
0.3706359254622106
0.37006177127340206
0.36948957740513455
0.3689193306453866
0.36835101774189544
0.36778462658799016
0.3672201463868164
0.3666575600126808
0.36609685

0.22394661328379498
0.22370965805635423
0.2234732609983159
0.22323741942490094
0.22300213153655984
0.22276739806616946
0.2225332103824223
0.22229956775807524
0.22206647273125113
0.2218339115958109
0.22160189444430084
0.2213704071165163
0.2211394586244009
0.22090903499839892
0.22067914561170615
0.22044977671833246
0.22022093463602124
0.219992611579506
0.21976480969518347
0.21953752421012235
0.21931075134933237
0.2190844948928116
0.21885874128164745
0.21863350324113384
0.21840876331584416
0.21818453115236547
0.21796080107900956
0.21773756756171975
0.21751483545899677
0.21729259714271715
0.21707084977062055
0.21684959473857118
0.21662882780148562
0.21640854967703627
0.21618875685768138
0.21596944625650455
0.2157506175467283
0.2155322685435777
0.21531439789282159
0.21509699950677436
0.21488007713531876
0.21466362588295418
0.2144476440030359
0.21423213072253958
0.21401708366032227
0.2138025024493757
0.21358838204925773
0.213374723177014
0.21316152310315173
0.21294878145829355
0.212736494561

In [3]:
### predict with pretrained weights, still need to import data

# create the network
nn_model_2 = NN(X_train, Y_train)
nn_model_2.add_layer(Layer(50, activation='relu'))
nn_model_2.add_layer(Layer(25, activation='sigmoid'))

# plot cost function
# Y_train_pred_2 = nn_model_2.predict_pretrained_weights(X_train)
Y_test_pred_2 = nn_model_2.predict_pretrained_weights(X_test)

# accuracy_2_train = np.sum(Y_train_pred_2 == Y_train) / len(Y_train) * 100
# print("Train accuracy 2: " + str(accuracy_2_train) + "%")
accuracy_2 = np.sum(Y_test_pred_2 == Y_test) / len(Y_test) * 100
print("Test accuracy 2: " + str(accuracy_2) + "%")

Test accuracy 2: 98.89705882352942%


In [4]:
### link
# https://stackoverflow.com/questions/37177811/crop-rectangle-returned-by-minarearect-opencv-python

def crop_minAreaRect(rect, box, gray, frame):
    mult = 1 # I wanted to show an area slightly larger than my min rectangle set this to one if you don't
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv.getRectSubPix(gray, size, center)    
    cropped = cv.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
#     cv.circle(frame, center, 10, (0,255,0), -1) # again this was mostly for debugging purposes
#     cv.rectangle(frame, 
#                  (int(center[0] - croppedW/2), int(center[1] - croppedH/2)), 
#                  (int(center[0] + croppedW/2), int(center[1] + croppedH/2)), 
#                  (255, 255, 255), 
#                  2) # again this was mostly for debugging purposes
    
    return croppedRotated

In [16]:
### test with video

backSub = cv.createBackgroundSubtractorKNN()
capture = cv.VideoCapture('12.mp4')
fps = 30
MIN_AREA = 600

if not capture.isOpened:
    print('Unable to open the video')
    exit(0)

while True:
    ret, frame = capture.read()
    if frame is None:
        break

    # convert to gray + resize + apply filter
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray_copy = gray.copy()
    gray = cv.GaussianBlur(gray, (21, 21), 0)

    fgMask = backSub.apply(gray)
    fgMask = cv.medianBlur(fgMask,5)
    
    # get contours
    cnts = cv.findContours(
        fgMask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    for c in cnts:
        rect = cv.minAreaRect(c)
        if (rect[1][0]*rect[1][1]) < MIN_AREA:
            continue
        
        box = cv.boxPoints(rect)
        box = np.int0(box)
        cv.drawContours(frame,[box],0,(0,255,0),2)
        
        img_crop = crop_minAreaRect(rect, box, gray_copy, frame)       
        img_crop = cv.resize(img_crop, (64,64), interpolation = cv.INTER_AREA)
        
        Y_box_pred = np.sum(nn_model_2.predict(img_crop.reshape(1, -1)))
        
        if Y_box_pred == 1:
            cv.putText(frame, "Person", (box[2][0], box[2][1]),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        if Y_box_pred == 0:
            cv.putText(frame, "Non", (box[2][0], box[2][1]),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        
        
        
        
#         if cv.contourArea(c) < MIN_AREA:
#             continue

#         (x, y, w, h) = cv.boundingRect(c)
#         cut_gray = gray_copy[y:y+h,x:x+w]
#         cut_gray = cv.resize(cut_gray, (64,64), interpolation = cv.INTER_AREA)
#         Y_box_pred = np.sum(nn_model_2.predict(cut_gray.reshape(1, -1)))
#         if Y_box_pred == 1:
#             cv.putText(frame, "Person", (x, y-10),
#                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
#         if Y_box_pred == 0:
#             cv.putText(frame, "Non", (x, y-10),
#                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
#         cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  
        
    cv.imshow('Frame', frame)
#     cv.imshow('FG Mask', fgMask)
#     cv.imshow('Gray copy', gray_copy)
    if cv.waitKey(int((1/fps)*1000)) & 0xFF == ord('q'):
        break

capture.release()
cv.destroyAllWindows()